In [ ]:
# Notebook: RAG with LangChain FAISS OpenAI
# Author: Thomas Purk
# Date: 2025-04-15
# Reference: https://platform.openai.com/docs/guides/conversation-state?api-mode=responses

# Multi-Turn Chat OpenAI API

Demonstrate multiple questions and response interation between a user and an LLM.

## Notebook Setup

In [1]:
!pip list | grep openai

openai                                1.72.0


In [39]:
# Setup the Notebook

# General
from google.colab import userdata
import openai

# User Inteface
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Load the OpenAI key from Colab"s Secrets
openai.api_key = userdata.get("OPENAI_API_KEY")

# Specify which OpenAI model to user
model_name = "gpt-4o-mini"

## Chat logic

In [50]:
# Define chat functions

def ask_chatbot(user_input):
    """ Wrapper function around OpenAI"s chat completions framework. Adds each
        user and LLM text to the conversations variable,

        Parameters:
            user_input (str): Contains the user"s prompt for the LLM.

        Returns:
            str: The LLM"s text response.

    """

    # Append user message
    conversation.append({"role": "user", "content": user_input})

    # Get response from OpenAI
    response = openai.chat.completions.create(
        model=model_name,
        messages=conversation
    )

    reply = response.choices[0].message.content

    # Append assistant reply to conversation
    conversation.append({"role": "assistant", "content": reply})

    return reply

def on_user_input(submit):
    """ Event handler for binding to the prompt entry text input on the
        user interface.

        Parameters:
            submit (ipywidgets.widgets.widget_string.Text): Event data.

    """

    # Disable controls while waiting for chat repsonse
    text_input.disabled = True
    with chat_history:
        clear_output()
        print("Waiting for Chat Response . . .")

    # Submit the User Prompt
    reply = ask_chatbot(submit.value)

    # Clear the input/output controls and print the repsonse
    submit.value = ""
    with chat_history:
        clear_output(wait=True)
        for item in conversation:
            display(Markdown(f"###{item['role'].upper()}\n---\n{item['content']}"))

    # The system is now ready for another user prompt
    text_input.disabled = False

In [51]:
# Create the user interface with ipwidgets

# Track the conversation
conversation = [
    {
        "role":"system", "content":"You are an experienced music teach with strong expertise in classical guitar performance"
    }
]

# Create the widget to capture the users question
text_input = widgets.Text(
    placeholder="Ask a quesion ...",
    description="You:",
    layout=widgets.Layout(width="600px")
)

# Create the widget to show the history of chats
chat_history = widgets.Output(
    layout={"border": "1px solid black", "height": "300px", "overflow_y": "auto"}
)

# Bind the callback to the user interface widget
text_input.on_submit(on_user_input)

#
chat_ui = widgets.VBox([chat_history, widgets.HBox([text_input])])
display(chat_ui)